<a href="https://colab.research.google.com/github/Sakthi1030/Resource-allocation-in-5g-network-using-KNN/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os


data_path = '/content/drive/MyDrive/KNN/R_S.csv'
if os.path.exists(data_path):
    print("File found.")
else:
    print("No such file or directory.")


In [ ]:
import pandas as pd
df = pd.read_csv(data_path)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def preprocess_and_classify_slices(df):

    df.replace('-', np.nan, inplace=True)

    df.dropna(axis=1, how='all', inplace=True)

    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

    df['NRxRSRP'] = pd.to_numeric(df['NRxRSRP'], errors='coerce')
    df['NRxRSRQ'] = pd.to_numeric(df['NRxRSRQ'], errors='coerce')

    df['NRxRSRP'].fillna(df['NRxRSRP'].mean(), inplace=True)
    df['NRxRSRQ'].fillna(df['NRxRSRQ'].mean(), inplace=True)

    df['Network_Quality'] = df.apply(lambda row: (row['NRxRSRP'] + row['NRxRSRQ']) / 2, axis=1)

    df.drop(columns=['Timestamp', 'Operatorname', 'NetworkMode', 'RSSI',
                     'State', 'CELLHEX', 'NODEHEX', 'LACHEX'], inplace=True)


    quantiles = df['Network_Quality'].quantile([0.33, 0.66])
    bins = [-float('inf'), quantiles[0.33], quantiles[0.66], float('inf')]
    labels = ['Low', 'Medium', 'High']
    df['Network_Slice_Class'] = pd.cut(df['Network_Quality'], bins=bins, labels=labels)

    print(df['Network_Slice_Class'].value_counts())
    return df

def plot_classification_results(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Low', 'Medium', 'High'], yticklabels=['Low', 'Medium', 'High'])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()


def plot_bandwidth_allocation(df):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Network_Slice_Class', y='Allocated_Bandwidth', data=df)
    plt.title("Bandwidth Allocation by Network Slice Class")
    plt.xlabel("Network Slice Class")
    plt.ylabel("Allocated Bandwidth (MBps)")
    plt.show()


df_static = pd.read_csv('/content/drive/MyDrive/KNN/R_S1.csv')
df_driving = pd.read_csv('/content/drive/MyDrive/KNN/R_D.csv')

df_static_processed = preprocess_and_classify_slices(df_static)
df_driving_processed = preprocess_and_classify_slices(df_driving)

df_static_processed['Allocated_Bandwidth'] = df_static_processed['Network_Slice_Class'].map({'Low': 50, 'Medium': 100, 'High': 150})
df_driving_processed['Allocated_Bandwidth'] = df_driving_processed['Network_Slice_Class'].map({'Low': 60, 'Medium': 120, 'High': 180})


plot_bandwidth_allocation(df_static_processed)
plot_bandwidth_allocation(df_driving_processed)

X_static = df_static_processed.drop(columns=['Network_Quality', 'Network_Slice_Class', 'Allocated_Bandwidth'])
y_static = df_static_processed['Network_Slice_Class']

X_driving = df_driving_processed.drop(columns=['Network_Quality', 'Network_Slice_Class', 'Allocated_Bandwidth'])
y_driving = df_driving_processed['Network_Slice_Class']


scaler = StandardScaler()
X_static = scaler.fit_transform(X_static)
X_driving = scaler.transform(X_driving)

X_train_static, X_test_static, y_train_static, y_test_static = train_test_split(X_static, y_static, test_size=0.2, random_state=42)
X_train_driving, X_test_driving, y_train_driving, y_test_driving = train_test_split(X_driving, y_driving, test_size=0.2, random_state=42)

param_grid = {
    'n_neighbors': np.arange(3, 20),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}


knn = KNeighborsClassifier()
grid_search_static = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search_static.fit(X_train_static, y_train_static)

print("Best parameters for static:", grid_search_static.best_params_)
print("Best score for static:", grid_search_static.best_score_)

y_pred_static = grid_search_static.best_estimator_.predict(X_test_static)
print("Static Environment Results:")
print(classification_report(y_test_static, y_pred_static))
plot_classification_results(y_test_static, y_pred_static, "Static Environment Confusion Matrix")

grid_search_driving = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search_driving.fit(X_train_driving, y_train_driving)

print("Best parameters for driving:", grid_search_driving.best_params_)
print("Best score for driving:", grid_search_driving.best_score_)

y_pred_driving = grid_search_driving.best_estimator_.predict(X_test_driving)
print("Driving Environment Results:")
print(classification_report(y_test_driving, y_pred_driving))
plot_classification_results(y_test_driving, y_pred_driving, "Driving Environment Confusion Matrix")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def generate_traffic_demand(df):
    np.random.seed(42)
    traffic_demand = np.random.rand(len(df))
    df['Traffic_Demand'] = traffic_demand
    return df

def dynamic_resource_allocation(df):

    base_bandwidth = {'Low': 50, 'Medium': 100, 'High': 150}
    df['Allocated_Bandwidth'] = df.apply(
        lambda row: base_bandwidth[row['Network_Slice_Class']] +
                    (row['Traffic_Demand'] * row['Network_Quality']), axis=1)

    return df


def visualize_dynamic_allocation(df):

    mean_bandwidth = df.groupby('Network_Slice_Class')['Allocated_Bandwidth'].mean()


    mean_bandwidth.plot(kind='bar', color='skyblue')
    plt.title('Dynamic Bandwidth Allocation by Network Slice Class')
    plt.ylabel('Allocated Bandwidth (Mbps)')
    plt.show()

df_static_processed = generate_traffic_demand(df_static_processed)

df_static_processed = dynamic_resource_allocation(df_static_processed)

visualize_dynamic_allocation(df_static_processed)

df_static_processed.to_csv('dynamic_resource_allocation.csv', index=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


rf = RandomForestClassifier()
rf.fit(X_train_static, y_train_static)
y_pred_rf_static = rf.predict(X_test_static)
print("Random Forest Static Accuracy:", accuracy_score(y_test_static, y_pred_rf_static))

svm = SVC()
svm.fit(X_train_static, y_train_static)
y_pred_svm_static = svm.predict(X_test_static)
print("SVM Static Accuracy:", accuracy_score(y_test_static, y_pred_svm_static))


In [ ]:
df